In [1]:
import numpy as np 
import pandas as pd 


In [3]:
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder


In [4]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [9]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split as tts 
X_train,X_test,y_train, y_test = tts(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [11]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [12]:
X_train

,age,gender,fever,cough,city
64,42,Male,104.0,Mild,Mumbai
29,34,Female,NaN,Strong,Mumbai
35,82,Female,102.0,Strong,Bangalore
45,72,Male,99.0,Mild,Bangalore
91,38,Male,NaN,Mild,Delhi
...,...,...,...,...,...
36,38,Female,101.0,Mild,Bangalore
2,42,Male,101.0,Mild,Delhi
83,17,Female,104.0,Mild,Kolkata
10,75,Female,NaN,Mild,Delhi


### AAM Zindagi

In [17]:
# We dont know how to use column transformer
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape


(80, 1)

In [18]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])

X_train_cough = oe.fit_transform(X_train[['cough']])

X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape


(80, 1)

In [24]:
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])


In [26]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values


In [27]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

### MENTOS Zindagi

In [28]:
from sklearn.compose import ColumnTransformer 


In [29]:
transformer = ColumnTransformer(transformers=[
        ('tnf1',SimpleImputer(),['fever']),
        ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
        ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
    ],remainder='passthrough')

In [31]:
transformer.fit_transform(X_train).shape

(80, 7)

In [32]:
transformer.transform(X_test).shape

(20, 7)